In [1]:
# youtube tutorial can be found at 
# https://www.youtube.com/watch?v=pveO4gwJ3_Y

# we will use very simple strategy, where we buy BTC 
# if the closing price is above the SMA_15 (15 seconds SMA)
# and we sell (fixing profit or loss) if 
# the closing price is below the SMA_15 (15 seconds SMA)


''' We will be using Binance socket to subscribe to real time data of BTCUSDT
'''


import websocket # !pip install websocket-client
import json
import pandas as pd

In [2]:
endpoint = 'wss://stream.binance.com:9443/ws'

In [3]:
our_msg = json.dumps({
    'method': 'SUBSCRIBE',
    'params': ['btcusdt@ticker'],
    'id': 1
})

In [6]:
df = pd.DataFrame()
in_position = False  # wether we are in a trade now
buyorders, sellorders = [], []   # lists of all trades
cumulative_profit = 0

def on_open(ws):
    ws.send(our_msg)
    

def on_message(ws, message):
    global df, in_position, buyorders, sellorders, cumulative_profit
    
    sma_period = 15
        
    out = json.loads(message)
    out = pd.DataFrame({'price':float(out['c'])},
                       index = [pd.to_datetime(out['E'], unit = 'ms')]                    
                      )  # storing only close prices into DataFrame
    df = pd.concat([df, out], axis = 0)
    # print(df)
    df = df.tail(50)    # restricting the maximum size of our data
                        # we keep only the number of ticks that we need for our strategy
    last_price = df.tail(1).price.values[0]
    sma = df.price.rolling(sma_period).mean().tail(1).values[0]   
        
    # Traiding logic. Buy signal:
    if not in_position and last_price > sma:
        print('Bought for ' + str(last_price))
        buyorders.append(last_price)
        in_position = True
                
    # Sell signal:
    if in_position and last_price < sma:
        print('Sold for ' + str(last_price))
        print('Profit: ', str(last_price - buyorders[-1]))
        cumulative_profit += last_price - buyorders[-1]
        print('Cumulative profit: ', cumulative_profit)
        print('')
        sellorders.append(last_price)
        in_position = False
        

In [7]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)

ws.run_forever()

Bought for 26403.28
Sold for 26403.28
Profit:  0.0
Cumulative profit:  0.0

Bought for 26403.29
Sold for 26403.28
Profit:  -0.010000000002037268
Cumulative profit:  -0.010000000002037268

Bought for 26396.4
Sold for 26404.58
Profit:  8.180000000000291
Cumulative profit:  8.169999999998254

Bought for 26404.59
Sold for 26412.01
Profit:  7.419999999998254
Cumulative profit:  15.589999999996508

Bought for 26404.59
Sold for 26404.58
Profit:  -0.00999999999839929
Cumulative profit:  15.579999999998108

Bought for 26404.59
Sold for 26404.58
Profit:  -0.00999999999839929
Cumulative profit:  15.569999999999709

Bought for 26404.59
Sold for 26404.58
Profit:  -0.00999999999839929
Cumulative profit:  15.56000000000131

Bought for 26404.59
Sold for 26404.58
Profit:  -0.00999999999839929
Cumulative profit:  15.55000000000291

Bought for 26404.59
Sold for 26404.58
Profit:  -0.00999999999839929
Cumulative profit:  15.540000000004511

Bought for 26404.59
Sold for 26404.58
Profit:  -0.0099999999983992

False